In [1]:
# make variables display whenever they are on their own line (not just the last line of a cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
import chess
import chess.pgn
import math

import numpy as np
import tensorflow as tf

In [3]:
def get_boards_from_game(game):
    boards = []
    node = game
    while not node.is_end():
        next_node = node.variations[0]
        node = next_node
        boards.append(node.board())
    return boards

In [4]:
def board_to_arr(board):
    color_to_num = {chess.WHITE: 1, chess.BLACK: -1}
    arr = np.zeros((8,8,6))
    
    for color in [True, False]:
        for piece in range(6):
            sqs = board.pieces(piece + 1, color)
            for sq in sqs:
                file = chess.square_file(sq)
                rank = chess.square_rank(sq)
                arr[file, rank, piece] = color_to_num[color]
    return arr

In [5]:
pgnf = open("data/ficsgamesdb_201701_standard_nomovetimes_1482388.pgn")
X = []
for i in range(10):
    game = chess.pgn.read_game(pgnf)
    boards = get_boards_from_game(game)
    for board in boards:
        X.append(board_to_arr(board))
X = np.array(X)

In [10]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
sess = tf.Session()

node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ",sess.run(node3))


node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


In [12]:
def create(x, layer_sizes):
    # Build the encoding layers
    next_layer_input = x

    encoding_matrices = []
    for dim in layer_sizes:
        input_dim = int(next_layer_input.get_shape()[1])

        # Initialize W using random values in interval [-1/sqrt(n) , 1/sqrt(n)]
        W = tf.Variable(tf.random_uniform([input_dim, dim], -1.0 / math.sqrt(input_dim), 1.0 / math.sqrt(input_dim)))

        # Initialize b to zero
        b = tf.Variable(tf.zeros([dim]))

        # We are going to use tied-weights so store the W matrix for later reference.
        encoding_matrices.append(W)

        output = tf.nn.tanh(tf.matmul(next_layer_input,W) + b)

        # the input into the next layer is the output of this layer
        next_layer_input = output

    # The fully encoded x value is now stored in the next_layer_input
    encoded_x = next_layer_input

    # build the reconstruction layers by reversing the reductions
    layer_sizes.reverse()
    encoding_matrices.reverse()


    for i, dim in enumerate(layer_sizes[1:] + [ int(x.get_shape()[1])]) :
        # we are using tied weights, so just lookup the encoding matrix for this step and transpose it
        W = tf.transpose(encoding_matrices[i])
        b = tf.Variable(tf.zeros([dim]))
        output = tf.nn.tanh(tf.matmul(next_layer_input,W) + b)
        next_layer_input = output

    # the fully encoded and reconstructed value of x is here:
    reconstructed_x = next_layer_input

    return {
    'encoded': encoded_x,
    'decoded': reconstructed_x,
    'cost' : tf.sqrt(tf.reduce_mean(tf.square(x-reconstructed_x)))
    }


In [30]:
sess = tf.Session()
x = tf.placeholder("float", [100, 8 * 8 * 6])
autoencoder = create(x, [60, 30, 20])
init = tf.global_variables_initializer()
sess.run(init)
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(autoencoder['cost'])


# Our dataset consists of two centers with gaussian noise w/ sigma = 0.1
c1 = np.array([0,0,0.5,0])
c2 = np.array([0.5,0,0,0])

# do 1000 training steps
for i in range(5000):
    sess.run(train_step, feed_dict={x: np.reshape(X[:100], (100, -1))})
    if i % 500 == 0:
        print(i, " cost", sess.run(autoencoder['cost'], feed_dict={x: np.reshape(X[:100], (100, -1))}))

0  cost 0.274056
500  cost 0.203873
1000  cost 0.169389
1500  cost 0.160678
2000  cost 0.146135
2500  cost 0.133144
3000  cost 0.130061
3500  cost 0.126469
4000  cost 0.121531
4500  cost 0.116807


In [31]:
encoded = sess.run(autoencoder['encoded'], feed_dict={x: np.reshape(X[:100], (100, -1))})

In [32]:
encoded.shape

(100, 20)